# Daily web scrape of stock prices
This code accesses site to retrieve stock price information daily ad add it to the h5 file. The steps are:
1. Get FTSE 100 and FTSE 250 lists from wikipedia, then compile into one list.
2. open pipe to h5 file holding current price data.
3. Loop though the tickers and for each one; 
    1. find the most recent price date.
    2. convert this into a time stamp to be used on Yahoo finance.
    3. go to Yahoo Finance and get all the prices between the last time stamp and the current timestamp.
4. Add these new prices to the h5 file.
5. Create a new (empty) h5 file and transfer all the data from the old file into the new one.
6. Delete the old h5 file and rename the new one.

The sources of data are:
- https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/indices/summary/summary-indices-constituents.html?index=NMX&page=1 -> list of FTSE 350 company stock tickers
- https://finance.yahoo.com/quote/{stock-ticker}/history?period1={start-time-mark}&period2={end-time-mark}&interval={interval}&filter=history&frequency={frequency} -> Example web address to retrieve information from Yahoo finance
    - Data on this page is scroll loaded so many time indexes must be used toretrieve the dcorrect data
    - Up to 145 records can be seen from initial page load note that this includes dividends so limit to 140 for safety

The inputs required for scrapping are:
 - {stock-ticker} -> this is the ticker taken from wiki with '.L' appended to it
 - {start-tme-mark} -> This is the time in seconds since 01/01/1970 at which you would like the data retrieval to start, data retrieved is inclusive of this time
 - {end-tme-mark} -> This is the time in seconds since 01/01/1970, data retrieved is inclusive of this time
 - {interval} & {frequency} -> This is the interval for which values are given, the two must match
     - 1d = 1 every 1 days
     - 1wk = 1 every week
     - 1mo = 1 eveery month

In [21]:
#Import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as rq
import re
import datetime as dt
import os
import tables
from rf_modules import *

# Delete the old temporary files (if they exist)

In [22]:
#close any open h5 files
tables.file._open_files.close_all()

In [23]:
#Delete the old h5 files
src_fldr_pth = r'C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\'
try:
    os.remove(src_fldr_pth + r'all_hist_prices_d_TMP.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_d_TMP.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.remove(src_fldr_pth + r'all_hist_prices_w_TMP.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_w_TMP.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))


ERROR - REMOVING:[WinError 2] The system cannot find the file specified: 'C:\\\\Users\\\\Robert\\\\Documents\\\\python_scripts\\\\stock_trading_ml_modelling\\\\historical_prices\\\\all_hist_prices_d_TMP.h5'

ERROR - REMOVING:[WinError 2] The system cannot find the file specified: 'C:\\\\Users\\\\Robert\\\\Documents\\\\python_scripts\\\\stock_trading_ml_modelling\\\\historical_prices\\\\all_hist_prices_w_TMP.h5'


# Scraping tickers
This section will scrap the ticker values for the FTSE 100 and FTSE 250 and store them in dataframes 'tick_ftse100' and 'tick_ftse250'.

Finally concatenate into 1 dataframe 'tick_ftse'.

In [24]:
#Fetch the data for ftse 100
web_add = r'https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/indices/summary/summary-indices-constituents.html?index=UKX&page=1'
resp = rq.get(web_add)
parser = bs(resp.content,'html.parser')
#Find how many pages there are to collect
par_elem = parser.find_all('div',id='pi-colonna1-display')[0]
num_pages = int(re.sub('[^0-9]','',par_elem.find_all('p')[0].text[-3:]))

#Collect the rows of data
row_li = []
for page in range(1,num_pages+1):
    web_add = r'https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/indices/summary/summary-indices-constituents.html?index=UKX&page={}'.format(page)
    resp = rq.get(web_add)
    parser = bs(resp.content,'html.parser')
    #Find how many pages there are to collect
    par_elem = parser.find_all('div',id='pi-colonna1-display')[0]
    #Collect the table
    table = par_elem.find_all('table')[0]
    #Collect the rows of data
    for row in table.tbody.find_all('tr'):
        temp_row = []
        for cell in row.find_all('td')[:2]:
            temp_row.append(re.sub('\n','',cell.text.upper()))
        row_li.append(temp_row)
print('count -> {}'.format(len(row_li)))
#Create a dataframe
tick_ftse100 = pd.DataFrame(data=row_li,columns=['ticker','company'])
tick_ftse100['index'] = 'FTSE100'
tick_ftse100.head()

count -> 101


,ticker,company,index
0,III,3I GRP.,FTSE100
1,ABF,A.B.FOOD,FTSE100
2,ADM,ADMIRAL GRP,FTSE100
3,AAL,ANGLO AMERICAN,FTSE100
4,ANTO,ANTOFAGASTA,FTSE100


In [25]:
#Fetch the data for ftse 250
web_add = r'https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/indices/summary/summary-indices-constituents.html?index=MCX&page=1'
resp = rq.get(web_add)
parser = bs(resp.content,'html.parser')
#Find how many pages there are to collect
par_elem = parser.find_all('div',id='pi-colonna1-display')[0]
num_pages = int(re.sub('[^0-9]','',par_elem.find_all('p')[0].text[-3:]))

#Collect the rows of data
row_li = []
for page in range(1,num_pages+1):
    web_add = r'https://www.londonstockexchange.com/exchange/prices-and-markets/stocks/indices/summary/summary-indices-constituents.html?index=MCX&page={}'.format(page)
    resp = rq.get(web_add)
    parser = bs(resp.content,'html.parser')
    #Find how many pages there are to collect
    par_elem = parser.find_all('div',id='pi-colonna1-display')[0]
    #Collect the table
    table = par_elem.find_all('table')[0]
    #Collect the rows of data
    for row in table.tbody.find_all('tr'):
        temp_row = []
        for cell in row.find_all('td')[:2]:
            temp_row.append(re.sub('\n','',cell.text.upper()))
        row_li.append(temp_row)
print('count -> {}'.format(len(row_li)))
#Create a dataframe
tick_ftse250 = pd.DataFrame(data=row_li,columns=['ticker','company'])
tick_ftse250['index'] = 'FTSE250'
tick_ftse250.head()

count -> 250


,ticker,company,index
0,3IN,3I INF. ORD,FTSE250
1,FOUR,4IMPRINT GRP.,FTSE250
2,ASL,ABERFTH.SMLL.CO,FTSE250
3,AGK,AGGREKO,FTSE250
4,AAF,AIRTEL AFRICA,FTSE250


In [26]:
#Combine into 1 dataframe
tick_ftse = pd.concat([tick_ftse100,tick_ftse250])
print('shape -> {}'.format(tick_ftse.shape))
print(tick_ftse.ticker.value_counts())
tick_ftse.sort_values(['ticker'])

shape -> (351, 3)
AVST    1
SNN     1
CRH     1
TALK    1
GSK     1
       ..
RR.     1
SXS     1
WWH     1
SHI     1
GVC     1
Name: ticker, Length: 351, dtype: int64


,ticker,company,index
0,3IN,3I INF. ORD,FTSE250
4,AAF,AIRTEL AFRICA,FTSE250
3,AAL,ANGLO AMERICAN,FTSE100
1,ABF,A.B.FOOD,FTSE100
2,ADM,ADMIRAL GRP,FTSE100
3,AGK,AGGREKO,FTSE250
10,AGR,ASSURA,FTSE250
13,AGT,AVI GLOBAL TST,FTSE250
5,AHT,ASHTEAD GRP.,FTSE100
5,AJB,AJ BELL,FTSE250


# Get the latest price dates
Get the latest price file.

Loop through the tickers in tick_ftse and for each one get the latest date of scrape.

Convert this date into a timestamp and add to the dataframe.

In [27]:
#open the price file
hist_prices_df = pd.read_hdf(src_fldr_pth + r'all_hist_prices_d.h5')
hist_prices_df.head()

,ticker,date,open,close,high,low,change,volume,ema12,ema26,macd_line,signal_line,macd
0,III,2008-01-02,995.0,1005.0,1023.0,995.0,10.0,785944.0,NaN,NaN,NaN,NaN,NaN
1,III,2008-01-03,1005.0,988.0,1020.0,981.0,-17.0,1805560.0,NaN,NaN,NaN,NaN,NaN
2,III,2008-01-04,985.5,965.0,1000.0,964.0,-20.5,1920061.0,NaN,NaN,NaN,NaN,NaN
3,III,2008-01-07,967.5,971.5,974.0,953.0,4.0,3167216.0,NaN,NaN,NaN,NaN,NaN
4,III,2008-01-08,975.0,978.5,989.0,971.5,3.5,2297118.0,NaN,NaN,NaN,NaN,NaN


In [29]:
#Convert date
def conv_date(_str_in):
    if type(_str_in) == str:
        return dt.datetime.strptime(_str_in,'%Y-%m-%d')
    else:
        return _str_in
hist_prices_df.date = [conv_date(x) for x in hist_prices_df.date]
hist_prices_df.dtypes

ticker                 object
date           datetime64[ns]
open                  float64
close                 float64
high                  float64
low                   float64
change                float64
volume                float64
ema12                 float64
ema26                 float64
macd_line             float64
signal_line           float64
macd                  float64
dtype: object

In [30]:
# hist_prices_df = hist_prices_df[hist_prices_df.date < '2019-09-30']
# hist_prices_df[hist_prices_df.ticker == 'SBRY'].date.value_counts()

1999-08-23    1
2001-08-03    1
2006-05-16    1
2010-12-09    1
2013-03-22    1
             ..
2017-09-07    1
1995-09-19    1
1997-12-31    1
2001-06-04    1
2019-08-12    1
Name: date, Length: 6032, dtype: int64

# Scraping a stock history
Working backwards through time from now until the last price scrape date collect all the daily data for a stock. Daily is used as it can later be summarised into weekly or monthly if required.

This will then be put into a dataframe containing:
- ticker
- Company name
- date
- open
- high
- low
- close
- Adjusted close (for divs and splits)
- volume

In [31]:
#Create a list of time intervals to be used with 140 days in each item
def create_sec_ref_li(_st_date:int,_en_date:int):
    #Establish the day ref of the dates compared to 01/01/1970
    _ep_date = pd.to_datetime(dt.datetime(1970,1,1),errors='coerce')
    _en_date = pd.to_datetime(_en_date,errors='coerce')
    _st_date = pd.to_datetime(_st_date,errors='coerce')
    print('_st_date: ' + str(_st_date))
    print('_en_date: ' + str(_en_date))
    _st_days = (_st_date - _ep_date).days
    _en_days = (_en_date - _ep_date).days
    #Loop adding to a list until reaching 0
    _sec_ref_li = []
    _days = 140
    while _en_days > _st_days:
        if _en_days - _days > _st_days:
            _sec_ref_li.append([(_en_days - _days)*86400,_en_days*86400])
        else:
            _sec_ref_li.append([_st_days*86400,_en_days*86400])        
        _en_days += -_days
    return _sec_ref_li

In [32]:
def calc_ema(s_in,periods):
    #Calc mod val
    mod = 2/(periods+1)
    #Make a df
    tmp_df = pd.DataFrame(s_in)
    #Calc sma
    tmp_df['sma'] = pd.Series([0] * len(s_in))
    for i in range(0,periods):
        tmp_df['sma'] += tmp_df[s_in.name].shift(i) / periods
    #Calc ema
    tmp_df['ema'] = tmp_df['sma'].copy()
    for row in tmp_df.iterrows():
        i = row[0]
        if i > 0 and not np.isnan(tmp_df['sma'][i-1]):
            tmp_df['ema'][i] = mod*(tmp_df[s_in.name][i] - tmp_df['ema'][i-1]) + tmp_df['ema'][i-1]
    return tmp_df['ema'].copy()

In [33]:
def calc_macd(ema_lng_s,ema_sht_s,sig_period):
    #Make a df
    tmp_df = pd.DataFrame([])
    tmp_df['ema_lng'] = ema_lng_s
    tmp_df['ema_sht'] = ema_sht_s
    #Calc the signal line
    tmp_df['macd_line'] = tmp_df['ema_sht'] - tmp_df['ema_lng']
    tmp_df['signal_line'] = calc_ema(tmp_df['macd_line'],sig_period)
    tmp_df['macd_hist'] = tmp_df['macd_line'] - tmp_df['signal_line']
    return (tmp_df['macd_line'].copy(),tmp_df['signal_line'].copy(),tmp_df['macd_hist'].copy())

In [34]:
#Create a dictionary of max character lengths of fields for use later in h5 file appending
def get_col_lens(_df_in):
    _col_lens = {}
    for c in _df_in:
        _tmp_s = pd.Series([len(str(x)) for x in _df_in[c]])
        _col_lens[c] = _tmp_s.max()
    return _col_lens
col_lens = get_col_lens(hist_prices_df)
# col_lens = {'ticker': 4,
#  'date': 19,
#  'open': 8,
#  'close': 8,
#  'high': 8,
#  'low': 8,
#  'change': 22,
#  'volume': 12,
#  'ema12': 18,
#  'ema26': 18,
#  'macd_line': 23,
#  'signal_line': 23,
#  'macd': 23}
col_lens

{'ticker': 4,
 'date': 19,
 'open': 8,
 'close': 8,
 'high': 8,
 'low': 8,
 'change': 22,
 'volume': 13,
 'ema12': 18,
 'ema26': 18,
 'macd_line': 23,
 'signal_line': 23,
 'macd': 23}

In [35]:
#Get the price history for a specific ticker
def get_price_hist_d(_tick:str,_sec_ref_li:list):
    try:
        _tick = re.sub('[^A-Z0-9\-.]','',_tick)
        print('Getting DAILY prices for:{}'.format(_tick))
        _tick_df = pd.DataFrame([])
        _cols = []

        for _secs in _sec_ref_li:
            try:
                _web_add = 'https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval={3}&filter=history&frequency={3}'.format(re.sub('\.','-',_tick)+'.L',_secs[0],_secs[1],'1d')
#                 print('_web_add: {}'.format(_web_add))
                
                _resp = rq.get(_web_add)
                if int(_resp.status_code) != 200:
                    print('status code: {}'.format(_resp.status_code))
                    continue
                _parser = bs(_resp.content,'html.parser')
                #Get the table
                _table = _parser.find_all('table',attrs={'data-test':'historical-prices'})[0]
                #Grab the data rows
                _rows = _table.find_all('tbody')[0].find_all('tr')
                #Put the rows into the dataframe
                for _r in _rows:
                    if len(_tick_df) == 0:
                        _cols = [clean_col_name(x.text) for x in _table.find_all('th')]
                        _tick_df = pd.DataFrame([],columns=_cols)
                    if len(_r.find_all('td')) == len(_cols):
                        _tick_df = _tick_df.append(pd.Series([x.text for x in _r.find_all('td')],index=_cols),ignore_index=True)
                    else:
                        continue
            except Exception as e:
                print('ERROR - CONTINUE:{}'.format(e))
                continue
        #Check for rows - if none then return
        if len(_tick_df) == 0:
            return _tick_df
        #Reformat
        def float_format(_str_in):
            if type(_str_in) == str:
                _str_in = _str_in.strip()
                _str_in = re.sub('[^0-9.]','',_str_in)
                if _str_in == '':
                    _str_in = 0
                return _str_in
            else:
                return _str_in
        _tick_df.loc[:,'open'] = _tick_df.loc[:,'open'].apply(float_format).astype(float)
        _tick_df.loc[:,'high'] = _tick_df.loc[:,'high'].apply(float_format).astype(float)
        _tick_df.loc[:,'low'] = _tick_df.loc[:,'low'].apply(float_format).astype(float)
        _tick_df.loc[:,'close'] = _tick_df.loc[:,'close'].apply(float_format).astype(float)
        _tick_df.loc[:,'adj_close'] = _tick_df.loc[:,'adj_close'].apply(float_format).astype(float)
        _tick_df.loc[:,'volume'] = _tick_df.loc[:,'volume'].apply(float_format).astype(float)
        _tick_df.loc[:,'change'] = _tick_df.loc[:,'close'] - _tick_df.loc[:,'open']
        def conv_date(_str_in):
            if type(_str_in) == str:
                return dt.datetime.strptime(_str_in,'%b %d, %Y')
            else:
                return _str_in
        _tick_df.loc[:,'date'] = _tick_df.loc[:,'date'].apply(conv_date)
        #Add the ticker series
        _tick_df.loc[:,'ticker'] = _tick    
        _tick_df = _tick_df.loc[:,['ticker','date','open','close','high','low','change','volume']]
        #CLEANING - Remove any rows with zero volume
        _tick_df = _tick_df[_tick_df['volume'] > 0]
        #CLEANING - Copy row above where the change has been more than 90%
        _tick_df['cl_change'] = (_tick_df['close'] - _tick_df['close'].shift(1))/_tick_df['close'].shift(1)
        _check_s = _tick_df['cl_change'] < -0.9
        _tick_df.loc[_check_s,'open'] = _tick_df['open'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'close'] = _tick_df['close'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'high'] = _tick_df['high'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'low'] = _tick_df['low'].shift(-1).copy().loc[_check_s]
        _tick_df.drop(columns='cl_change',inplace=True)
        return _tick_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

#Calc the ema and macds for the data
def calc_ema_macd(_tick_df):
    try:
        #Add in the ema and macd
        _tick_df = _tick_df.sort_values(by='date')
        _tick_df = _tick_df.reset_index(drop=True)
        _tick_df['ema12'] = calc_ema(_tick_df['close'],12)
        _tick_df['ema26'] = calc_ema(_tick_df['close'],26)
        _tick_df['macd_line'],_tick_df['signal_line'],_tick_df['macd'] = calc_macd(_tick_df['ema26'],_tick_df['ema12'],9)
        #Sort clean and export
        _tick_df = _tick_df.sort_values(by='date')
        _tick_df = _tick_df.reset_index(drop=True)
        return _tick_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

In [36]:
#Create a weekly table
def get_price_hist_w(_df_d):
    print('Converting daily prices to weekly prices')
    try:
        #Create a copy of the data
        _df_d = _df_d.copy()
        #Establish a week number for each date
        _df_d['isocalendar'] = [x.isocalendar()[:2] for x in _df_d['date']]
        #Get highs and lows
        _high_df = _df_d.loc[_df_d['high'] > 0,['high','isocalendar']].groupby('isocalendar').max().reset_index()
        _low_df = _df_d.loc[_df_d['low'] > 0,['low','isocalendar']].groupby('isocalendar').min().reset_index()
        #Get total volume for the week
        _vol_df = _df_d.loc[_df_d['volume'] > 0,['volume','isocalendar']].groupby('isocalendar').sum().reset_index()
        #Get open price
        _min_wk_day = _df_d.loc[_df_d['open'] > 0,['date','isocalendar']].groupby('isocalendar').min().reset_index()
        _open_df = pd.merge(_df_d[['date','open']],_min_wk_day,left_on='date',right_on='date')
        #Get close price
        _max_wk_day = _df_d.loc[_df_d['close'] > 0,['date','isocalendar']].groupby('isocalendar').max().reset_index()
        _close_df = pd.merge(_df_d[['date','close']],_max_wk_day,left_on='date',right_on='date').reset_index()
        #Form the final df
        _wk_df = pd.merge(_df_d[['ticker','isocalendar']],_min_wk_day,left_on='isocalendar',right_on='isocalendar') #date
        _wk_df = pd.merge(_wk_df,_high_df,left_on='isocalendar',right_on='isocalendar') #high
        _wk_df = pd.merge(_wk_df,_low_df,left_on='isocalendar',right_on='isocalendar') #low
        _wk_df = pd.merge(_wk_df,_vol_df,left_on='isocalendar',right_on='isocalendar') #volume
        _wk_df = pd.merge(_wk_df,_open_df[['isocalendar','open']],left_on='isocalendar',right_on='isocalendar') #open
        _wk_df = pd.merge(_wk_df,_close_df[['isocalendar','close']],left_on='isocalendar',right_on='isocalendar') #close
        _wk_df['change'] = _wk_df['close'] - _wk_df['open']
        _wk_df = _wk_df.drop_duplicates().reset_index(drop=True)
        #Get the monday of each week
        _wk_df['weekday'] = [dt.date.weekday(x) for x in _wk_df['date']]
        _wk_df['date'] = _wk_df['date'] - pd.Series([dt.timedelta(days=x) for x in _wk_df['weekday']])
        _wk_df.drop(columns=['isocalendar','weekday'],inplace=True)
        return _wk_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

In [37]:
#Scrape daily price data
hf_store_name_d = src_fldr_pth + r'all_hist_prices_d_TMP.h5'
hf_d = pd.HDFStore(hf_store_name_d)
group_name_d = r'daily_data'
hf_store_name_w = src_fldr_pth + r'all_hist_prices_w_TMP.h5'
hf_w = pd.HDFStore(hf_store_name_w)
group_name_w = r'weekly_data'
out_cols = ['ticker','date','open','close','high','low','change','volume']
count = 0
errors = []
run_time = process_time()
for tick in tick_ftse['ticker']:
# for tick in ['SBRY']:
    try:
        run_time.show_latest_lap_time()
        run_time.lap()
        count += 1
        print('\n{} RUNNING FOR: {}'.format(count,tick))
        tick_df = hist_prices_df[hist_prices_df.ticker == tick][out_cols]
        print('DAILY SHAPE BEFORE: {}'.format(tick_df.shape))
        
        #DAILY PRICES
        #Get last scrapped date
        st_date = tick_df.date.max()+dt.timedelta(days=1) #Add 1 day so we don't keep grabbing the same day price
        if pd.isnull(st_date):
            #Treat as a new share
            st_date = dt.datetime(1970,1,1)
        
        #Get new price data
        new_tick_df = get_price_hist_d(tick,create_sec_ref_li(st_date,dt.datetime.today()+dt.timedelta(days=1))) #1 day ahead so today is included
        
        #Join onto existing data
        tick_df = tick_df.append(new_tick_df)
        
        #Drop duplicates
        tick_df = tick_df.drop_duplicates()

        #Calc emas and macd
        tick_df = calc_ema_macd(tick_df)
        print('DAILY FINAL SHAPE: {}'.format(tick_df.shape))
        
        #Clarify col_lens with cur cols in data
        col_lens_tmp = {}
        for col in tick_df:
            if col in col_lens:
                col_lens_tmp[col] = col_lens[col]
                
        #Add to daily h5 file
        tick_df.to_hdf(hf_store_name_d,key=group_name_d,append=True,min_itemsize=col_lens_tmp)
        
        #WEEKLY PRICES
        #Convert to weekly prices
        df_w = get_price_hist_w(tick_df)
        
        #Drop duplicates
        df_w = df_w.drop_duplicates()

        #Calc emas and macd
        df_w = calc_ema_macd(df_w)
        print('WEEKLY FINAL SHAPE: {}'.format(df_w.shape))

        #Add to weekly h5 file
        df_w.to_hdf(hf_store_name_w,key=group_name_w,append=True,min_itemsize=col_lens)
    except Exception as e:
        print('ERROR:{}'.format(e))
        errors.append(e)
hf_d.close()
hf_w.close()
print('\n\n')
run_time.end()
print('\nERROR COUNT: {}'.format(len(errors)))
if len(errors) > 0:
    print('    ERRORS -> {}'.format(errors))


1 RUNNING FOR: III
DAILY SHAPE BEFORE: (2945, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 17:51:16.394900
Getting DAILY prices for:III
DAILY FINAL SHAPE: (2948, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (613, 13)
LAP 1 TIME -> 0:0:0

2 RUNNING FOR: ABF
DAILY SHAPE BEFORE: (5013, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 17:51:23.881371
Getting DAILY prices for:ABF
DAILY FINAL SHAPE: (5018, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1045, 13)
LAP 2 TIME -> 0:0:7

3 RUNNING FOR: ADM
DAILY SHAPE BEFORE: (1667, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 17:51:32.549561
Getting DAILY prices for:ADM
DAILY FINAL SHAPE: (1672, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (353, 13)
LAP 3 TIME -> 0:0:8

4 RUNNING FOR: AAL
DAILY SHAPE BEFORE: (4284, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 17:51:36.398321
Getting DAILY prices for:AAL
DAILY FINAL SHAPE: (4289, 13)
Converting daily price

ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE

ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE

DAILY FINAL SHAPE: (5044, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1046, 13)
LAP 32 TIME -> 0:0:6

33 RUNNING FOR: FLTR
DAILY SHAPE BEFORE: (85, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 17:59:34.379183
Getting DAILY prices for:FLTR
DAILY FINAL SHAPE: (90, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (18, 13)
LAP 33 TIME -> 0:0:8

34 RUNNING FOR: FRES
DAILY SHAPE BEFORE: (2876, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 17:59:37.131086
Getting DAILY prices for:FRES
DAILY FINAL SHAPE: (2881, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (596, 13)
LAP 34 TIME -> 0:0:2

35 RUNNING FOR: GSK
DAILY SHAPE BEFORE: (4991, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 17:59:43.371548
Getting DAILY prices for:GSK
DAILY FINAL SHAPE: (4996, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1045, 13)
LAP 35 TIME -> 0:0:6

36 RUNNING FOR: GLEN
DAILY SHAPE BEFORE: (2115, 8)
_st_date: 2019-09-

LAP 42 TIME -> 0:0:9

43 RUNNING FOR: INF
DAILY SHAPE BEFORE: (5142, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:01:36.792355
Getting DAILY prices for:INF
DAILY FINAL SHAPE: (5147, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1066, 13)
LAP 43 TIME -> 0:0:8

44 RUNNING FOR: IHG
DAILY SHAPE BEFORE: (4067, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:01:45.682444
Getting DAILY prices for:IHG
DAILY FINAL SHAPE: (4072, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (844, 13)
LAP 44 TIME -> 0:0:8

45 RUNNING FOR: ITRK
DAILY SHAPE BEFORE: (4097, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:01:55.337704
Getting DAILY prices for:ITRK
DAILY FINAL SHAPE: (4102, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (884, 13)
LAP 45 TIME -> 0:0:9

46 RUNNING FOR: IAG
DAILY SHAPE BEFORE: (4131, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:02:05.141834
Getting DAILY prices for:IAG
DAILY FINAL SHAPE: (41

...

DAILY FINAL SHAPE: (1969, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (464, 13)
LAP 258 TIME -> 0:0:2

259 RUNNING FOR: NESF
DAILY SHAPE BEFORE: (1285, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:35:54.769510
Getting DAILY prices for:NESF
DAILY FINAL SHAPE: (1288, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (276, 13)
LAP 259 TIME -> 0:0:4

260 RUNNING FOR: OSB
DAILY SHAPE BEFORE: (1345, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:35:59.001635
Getting DAILY prices for:OSB
DAILY FINAL SHAPE: (1350, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (279, 13)
LAP 260 TIME -> 0:0:4

261 RUNNING FOR: OXIG
DAILY SHAPE BEFORE: (6784, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:36:03.124236
Getting DAILY prices for:OXIG
DAILY FINAL SHAPE: (6789, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1459, 13)
LAP 261 TIME -> 0:0:4

262 RUNNING FOR: PAGE
DAILY SHAPE BEFORE: (4124, 8)
_st_da

ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE

DAILY FINAL SHAPE: (2803, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (586, 13)
LAP 299 TIME -> 0:0:11

300 RUNNING FOR: SOI
DAILY SHAPE BEFORE: (2799, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:41:10.496005
Getting DAILY prices for:SOI
DAILY FINAL SHAPE: (2802, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (585, 13)
LAP 300 TIME -> 0:0:5

301 RUNNING FOR: SCIN
DAILY SHAPE BEFORE: (2805, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:41:18.930534
Getting DAILY prices for:SCIN
DAILY FINAL SHAPE: (2808, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (586, 13)
LAP 301 TIME -> 0:0:8

302 RUNNING FOR: SNR
DAILY SHAPE BEFORE: (6413, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:41:24.539619
Getting DAILY prices for:SNR
DAILY FINAL SHAPE: (6418, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1428, 13)
LAP 302 TIME -> 0:0:5

303 RUNNING FOR: SEQI
DAILY SHAPE BEFORE: (1084, 8)
_st_dat

LAP 329 TIME -> 0:0:6

330 RUNNING FOR: TPK
DAILY SHAPE BEFORE: (4946, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:44:43.710992
Getting DAILY prices for:TPK
DAILY FINAL SHAPE: (4951, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1026, 13)
LAP 330 TIME -> 0:0:3

331 RUNNING FOR: BBOX
DAILY SHAPE BEFORE: (1302, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:44:53.988531
Getting DAILY prices for:BBOX
DAILY FINAL SHAPE: (1305, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (276, 13)
LAP 331 TIME -> 0:0:10

332 RUNNING FOR: TLW
DAILY SHAPE BEFORE: (4837, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:44:58.193468
Getting DAILY prices for:TLW
DAILY FINAL SHAPE: (4842, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1018, 13)
LAP 332 TIME -> 0:0:4

333 RUNNING FOR: UDG
DAILY SHAPE BEFORE: (4251, 8)
_st_date: 2019-09-28 00:00:00
_en_date: 2019-10-05 18:45:06.537610
Getting DAILY prices for:UDG
DAILY FINAL 

ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE:list index out of range
ERROR - CONTINUE

In [38]:
#close any open h5 files
tables.file._open_files.close_all()

In [39]:
#Delete the old h5 file and rename the TMP
try:
    os.remove(src_fldr_pth + r'all_hist_prices_d.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_d.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.rename(src_fldr_pth + r'all_hist_prices_d_TMP.h5',src_fldr_pth + r'all_hist_prices_d.h5')
    print('\nSUCCESSFULLY RENAMED {} TO {}'.format(src_fldr_pth + r'all_hist_prices_d_TMP.h5',src_fldr_pth + r'all_hist_prices_d.h5'))
except Exception as e:
    print('\nERROR - RENAMING:{}'.format(e))


SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d.h5

SUCCESSFULLY RENAMED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d_TMP.h5 TO C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d.h5


In [40]:
#Delete the old h5 file and rename the TMP
try:
    os.remove(src_fldr_pth + r'all_hist_prices_w.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_w.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.rename(src_fldr_pth + r'all_hist_prices_w_TMP.h5',src_fldr_pth + r'all_hist_prices_w.h5')
    print('\nSUCCESSFULLY RENAMED {} TO {}'.format(src_fldr_pth + r'all_hist_prices_w_TMP.h5',src_fldr_pth + r'all_hist_prices_w.h5'))
except Exception as e:
    print('\nERROR - RENAMING:{}'.format(e))


SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w.h5

SUCCESSFULLY RENAMED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w_TMP.h5 TO C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w.h5


# Exporting the FTSE ticker

In [41]:
#Export the ftse list
path = "C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\"
tick_ftse.to_csv(path_or_buf=path + "tick_ftse.csv")

In [42]:
#End the program
import sys
sys.exit()

SystemExit: 

C:\Users\Robert\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
